# MS Calculations

Notebook to crunch numbers for the MS.

by Cascade Tuholske 2020.02.23 

Updated 2020.08.27 - CPT
Was run on ERA5 RH with CHIRTS-Daily Tmax from ERA5

In [1]:
#### Depdencies 
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import glob
import matplotlib.dates as mdates

In [2]:
#### Regressions, no intercept addition is needed because we're using SK LEARN HERE 

def lm_func(df, col):
    
    "simple linear model of a time series data, returns coef"
    
    # Get Data
    X_year = np.array(df.groupby('year')['ID_HDC_G0'].mean().index).reshape((-1, 1))
    Y_stats = np.array(df.groupby('year')[col].sum()).reshape((-1, 1))

    # Add Intercept
    X_year_2 = sm.add_constant(X_year)

    # Regress
    model = sm.OLS(Y_stats, X_year_2).fit() 
        
    coef = int(model.params[1])
    #coef = int(coef)
            
    # R2 and P
    r2 = model.rsquared_adj
    p = model.pvalues[0]
    
    return coef, round(r2, 2), round(p, 3)

In [3]:
#### Load Data
DATA = 'WBGT32_1D' # UPDATE 

# file paths
DATA_IN = "/home/cascade/projects/UrbanHeat/data/"  
FIG_OUT = "/home/cascade/projects/UrbanHeat/figures/"
FN_IN = 'processed/PNAS-DATA-v2/'+DATA+'_EXP.json'
HI_STATS = pd.read_json(DATA_IN+FN_IN, orient = 'split')

# Set scale
scale = 10**9


In [4]:
# Drop cites where 1983 had 1 day and none elsewhere

print(len(HI_STATS))
only83 = HI_STATS.groupby('ID_HDC_G0')['tot_days'].sum() == 1 # sum up total days and find those with 1 day
only83 = list(only83[only83 == True].index) # make a list of IDs
sub = HI_STATS[HI_STATS['ID_HDC_G0'].isin(only83)] # subset those IDs
bad_ids = sub[(sub['year'] == 1983) & (sub['tot_days'] == 1)] # drop those from 1983 only
drop_list = list(bad_ids['ID_HDC_G0']) # make a list
HI_STATS= HI_STATS[~HI_STATS['ID_HDC_G0'].isin(drop_list)] # drop those from the list
print(len(HI_STATS))

322082
321538


In [5]:
#### Add In Meta Data (e.g. geographic data)
meta_fn = DATA_IN+'interim/GHS-UCDB-IDS.csv'
meta_data = pd.read_csv(meta_fn)

#### Merge in meta
HI_STATS = HI_STATS.merge(meta_data, on = 'ID_HDC_G0', how = 'left')

In [7]:
#### Check to num days per year greatest > 32
check = HI_STATS.sort_values('tot_days', ascending = False)
check[['CTR_MN_NM', 'UC_NM_MN', 'year', 'tot_days']].head(5)

,CTR_MN_NM,UC_NM_MN,year,tot_days
215647,Indonesia,Pekan Baru [IDN],2016,228
67759,Saudi Arabia,Samitah [SAU],2002,223
67770,Saudi Arabia,Samitah [SAU],2013,222
217682,Indonesia,Telukkepik [IDN],2016,221
67843,Yemen,Harad [YEM],2002,220


# Global Trends

In [8]:
#### Total Change in people Days
data = HI_STATS.groupby('year')['people_days'].sum()
year = str(data.index[33])
value = str(data.values[33]/10**9)
print('person days in 2016 was '+value+' billion')

year = str(data.index[0])
value = str(data.values[0]/10**9)
print('person days in 1983 was '+value+' billion')

#### Pct Change in Poeple Days 1983 - 2016
pdays16 = data.iloc[len(data) -1]
pdays83 = data.iloc[0]
out = (data.iloc[len(data) -1] - data.iloc[0]) / data.iloc[0] * 100
print('pct increase in people days 83 - 16 is ', out)


person days in 2016 was 125.6137485348937 billion
person days in 1983 was 44.458472045749346 billion
pct increase in people days 83 - 16 is  182.54175808298746


In [9]:
#### Rate of change
data = HI_STATS
coef, r2, p = lm_func(data, 'people_days')
print('annual increase in people days ', 'was', coef/10**9, ' p=', p)
coef1, r21, p1 = lm_func(data, 'people_days_heat')
print('annual increase in people days heat ', 'was', coef1/10**9, ' p=', p)
coef2, r22, p2 = lm_func(data, 'people_days_pop')
print('annual increase in people days pop ', 'was', coef2/10**9, ' p=', p)
print('attrib heat ', 'was', coef1 / coef *100, ' p=', p, '\n')

annual increase in people days  was 2.217374507  p= 0.0
annual increase in people days heat  was 0.727456266  p= 0.0
annual increase in people days pop  was 1.48991824  p= 0.0
attrib heat  was 32.807099734551066  p= 0.0 



In [10]:
#### Pct Pday Annual Increase from Heat
coef_pdays, r2_pdays, p_pdays = lm_func(HI_STATS, 'people_days') # regress pdays
coef_heat, r2_heat, p_heat = lm_func(HI_STATS, 'people_days_heat') # regreas heat

print('warming is what pct of total?', coef_heat/coef_pdays *100)

warming is what pct of total? 32.807099734551066


In [11]:
#### Pct heat vs pop
coef_pop, r2_pdays, p_pdays = lm_func(HI_STATS, 'people_days_pop') # regress pdays
coef_heat, r2_heat, p_heat = lm_func(HI_STATS, 'people_days_heat') # regreas heat

print('pct of heat vs pop', coef_heat/coef_pop *100)

pct of heat vs pop 48.82524735048548


# City-level

#### Largest cities compared to global total

In [12]:
#### Top cities
cities = pd.read_csv(DATA_IN+'processed/PNAS-DATA-v2/WBGT32_1D_EXP-TOP50.csv')

In [13]:
top = cities.sort_values('coef_pdays', ascending = False).head(25) # get the top ten cities


In [17]:
# What pct of the global annual increase comes from the top 25 cities?
ans = top['coef_pdays'].sum() / coef # coef comes from rate of change cell
print('Top 25 cities of total annual increase', ans * 100)

Top 25 cities of total annual increase 24.507137475575398


#### Pdays

In [18]:
city_coefs = pd.read_json(DATA_IN+'processed/PNAS-DATA-v2/WBGT32_1D_TREND_EXP05.json', orient = 'split')
GHS = gpd.read_file('/home/cascade/projects/UrbanHeat/data/raw/GHS_UCDB/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_0.shp')

In [19]:
len(city_coefs)

6022

In [20]:
#### Number of cities w/ sig increase in exposure?
print('The pct of cities w/ increases in exposure: ', len(city_coefs)/len(GHS)*100)


The pct of cities w/ increases in exposure:  45.84697373429768


In [21]:
len(city_coefs)

6022

In [22]:
ans = len(city_coefs[(city_coefs['GCPNT_LAT'] < 23.5) & (city_coefs['GCPNT_LAT'] > -23.5)]) / len(city_coefs)
print('what pct of pday cities are low lat?', ans*100)

what pct of pday cities are low lat? 50.94652939222849


In [23]:
print('what pct of global pop are cities with sig pdays?')

what pct of global pop are cities with sig pdays?


In [24]:
def country_search(country, data_set):
    "what pct of cities had a p-day increase?"
    print('Num of Cities in '+country+' ', len(data_set[data_set['CTR_MN_NM'] == country]) / len(GHS[GHS['CTR_MN_NM'] == country]) *100)

In [25]:
data_set = city_coefs

In [26]:
country_search('Senegal', data_set)

Num of Cities in Senegal  93.93939393939394


In [27]:
country_search('Nigeria', data_set)

Num of Cities in Nigeria  91.92546583850931


In [28]:
country_search('India', data_set)

Num of Cities in India  86.48399014778325


#### Pct of global population exposured

In [29]:
city_coefs = pd.read_json(DATA_IN+'processed/PNAS-DATA-v2/WBGT32_1D_TREND_EXP05.json', orient = 'split')

In [30]:
pop = pd.read_csv(DATA_IN+'interim/GHS-UCDB-Interp.csv')
p16 = pop[['ID_HDC_G0', 'P2016']]

In [31]:
len(p16)

13135

In [32]:
pdays_pop = pd.merge(city_coefs[['ID_HDC_G0']], p16, on = 'ID_HDC_G0', how = 'inner')

In [33]:
ans = pdays_pop['P2016'].sum() / p16['P2016'].sum() * 100 
print('What is the global urban population in 2016', p16['P2016'].sum())
print('How many people live in cities with increasing exp in 2016', pdays_pop['P2016'].sum())
print('What pct of total urban pop has sig increase exp in 2015', ans)

What is the global urban population in 2016 3535326298.5424414
How many people live in cities with increasing exp in 2016 1661009084.2793088
What pct of total urban pop has sig increase exp in 2015 46.983190348345396


In [34]:
# From UN-DESA 2018 estimates for total global pop in 2015
ans =  pdays_pop['P2016'].sum() / 7383009000 * 100
print('What pct of total world pop has sig increase exp in 2016', ans)

What pct of total world pop has sig increase exp in 2016 22.49772530792403


In [35]:
# UN-DESA Urban pop in 2015 was  3 981 498
p16['P2016'].sum()

3535326298.5424414

#### Total Heat Days 

In [36]:
city_totdays = pd.read_json(DATA_IN+'processed/PNAS-DATA-v2/WBGT32_1D_TREND_HEATP05.json', orient = 'split')

In [37]:
print('What pct of all cities had sig increase in days/yr > WBGT32 C ?')
print(len(city_totdays)/len(GHS))
print(len(city_totdays))

What pct of all cities had sig increase in days/yr > WBGT32 C ?
0.38797106966121053
5096


In [38]:
print('What pct of all cities >1 day / yr in days/yr > WBGT32 C ?')
print(len(city_totdays[city_totdays['coef_totDays'] >= 1])/len(GHS))
print(len(city_totdays[city_totdays['coef_totDays'] >= 1]))

What pct of all cities >1 day / yr in days/yr > WBGT32 C ?
0.1884278644842025
2475


In [39]:
## How many cities day increase per year ... 1, 3
top = len(city_totdays)
bottom = len(city_totdays[city_totdays['coef_totDays'] >= 2])

In [40]:
print(top)
print(bottom)

5096
66


In [44]:
## What are some big cities (>1m people)?
hot1m = city_totdays[(city_totdays['coef_totDays'] >= 1.5) & (city_totdays['P2016'] >= 10**6)][['coef_totDays', 'UC_NM_MN', 'P2016']].sort_values('coef_totDays')

In [46]:
len(hot1m)

25

In [47]:
hot1m

,coef_totDays,UC_NM_MN,P2016
5777,1.506952,Dhaka [BGD],2.394235e+07
4872,1.519328,Patna [IND],2.771973e+06
2450,1.522231,Ad-Dammam [SAU],1.564432e+06
4871,1.528037,Muzaffarpur [IND],1.435602e+06
3830,1.529412,Hardoi [IND],1.185764e+06
3911,1.530634,Kanpur [IND],3.773461e+06
1580,1.543468,Maiduguri [NGA],1.069539e+06
1232,1.554927,Ihiala [NGA],1.713471e+06
2403,1.558136,Ahvaz [IRN],1.232269e+06
5824,1.565470,Brahman Bariya [BGD],1.874156e+06


#### Dehli & Kolkata 

In [49]:
# Delhi 6955 & Kolkata 9691
K = city_coefs[city_coefs['ID_HDC_G0']== 9691]
D = city_coefs[city_coefs['ID_HDC_G0']== 6955]

In [50]:
print('Share of heat Kolkata', K.coef_heat / K.coef_pdays * 100)

Share of heat Kolkata 5599    50.402206
dtype: float64


In [51]:
print('Share of heat Delhi', D.coef_heat / D.coef_pdays * 100)

Share of heat Delhi 3201    23.607115
dtype: float64


#### Populations of specific cities

In [52]:
pop = pd.read_csv(DATA_IN+'interim/GHS-UCDB-Interp.csv')

In [53]:
# 9691, Kolkata 1998
# 2046, Paris 2003
# 4417, Aleppo 2010

In [54]:
ans = pop[pop['ID_HDC_G0'] == 9691]['P2015'] / 10**3
print('Pop of Kolkata in 1998', ans)

Pop of Kolkata in 1998 7475    21620.289279
Name: P2015, dtype: float64


## WBGT 32 vs 28

In [55]:
wbgt32 = pd.read_json(DATA_IN+'processed/PNAS-DATA-v2/WBGT32_1D_TREND_EXP05.json', orient = 'split')
wbgt28 = pd.read_json(DATA_IN+'processed/PNAS-DATA-v2/WBGT28_1D_TREND_EXP05.json', orient = 'split')

In [56]:
print('how many cities wbgt 32?', len(wbgt32))
print('how many cities wbgt 28?', len(wbgt28))
print('no trend', len(meta_data) - 8510)
print('dif is', 8510 - 6022)

how many cities wbgt 32? 6022
how many cities wbgt 28? 8510
no trend 4625
dif is 2488


# Regional Trends

In [57]:
#### Annual Rates

scale = 10**6
geog = 'sub-region'

for label in np.unique(HI_STATS[geog]):
    label = label
    data = HI_STATS[HI_STATS[geog] == label]
    
    #### Rate of change
    coef, r2, p = lm_func(data, 'people_days')
    print('annual increase in people days '+label, 'was', coef/scale, ' p=', p)
    coef1, r21, p1 = lm_func(data, 'people_days_heat')
    print('annual increase in people days heat '+label, 'was', coef1/scale, ' p=', p)
    coef2, r22, p2 = lm_func(data, 'people_days_pop')
    print('annual increase in people days pop '+label, 'was', coef2/scale, ' p=', p)
    print('attrib heat '+label, 'was', coef1 / coef *100, ' p=', p, '\n')
  

annual increase in people days Australia and New Zealand was 0.145165  p= 0.09
annual increase in people days heat Australia and New Zealand was 0.062207  p= 0.09
annual increase in people days pop Australia and New Zealand was 0.082957  p= 0.09
attrib heat Australia and New Zealand was 42.85261598870251  p= 0.09 

annual increase in people days Central Asia was 0.097638  p= 0.576
annual increase in people days heat Central Asia was -0.059772  p= 0.576
annual increase in people days pop Central Asia was 0.15741  p= 0.576
attrib heat Central Asia was -61.21796841393719  p= 0.576 

annual increase in people days Eastern Asia was 130.729143  p= 0.0
annual increase in people days heat Eastern Asia was 34.095827  p= 0.0
annual increase in people days pop Eastern Asia was 96.633316  p= 0.0
attrib heat Eastern Asia was 26.081274777422813  p= 0.0 

annual increase in people days Eastern Europe was 0.132212  p= 0.339
annual increase in people days heat Eastern Europe was 0.098434  p= 0.339
annu

In [58]:
#### Trends for Africa, N & SS
geog = 'region'
location = 'Africa'
data = HI_STATS[HI_STATS[geog] == location]
print(location)

#### Total Change in people Days
data = data.groupby('year')['people_days'].sum()
year = str(data.index[33])
value = str(data.values[33]/10**9)
print('person days in 2016 was '+value+' billion')

year = str(data.index[0])
value = str(data.values[0]/10**9)
print('person days in 1983 was '+value+' billion')

#### Pct Change in Poeple Days 1983 - 2016
pdays16 = data.iloc[len(data) -1]
pdays83 = data.iloc[0]
out = (data.iloc[len(data) -1] - data.iloc[0]) / data.iloc[0] * 100
print('pct increase in people days 83 - 16 is ', out)



Africa
person days in 2016 was 13.220664862528546 billion
person days in 1983 was 3.0001575378206344 billion
pct increase in people days 83 - 16 is  340.6656882468999


In [59]:
#### S Asia as pct of total  global = 5.245146271 B 

print('pct of total pdays from S Asia is ', 1899.70765 / 10**3 / 5.245146271 * 100)

pct of total pdays from S Asia is  36.218392240142734


In [60]:
#### Median Slope
region = 'Europe'
col = 'coef_heat'
geog = 'region'
scale = 10**3
result = city_coefs[city_coefs[geog]== region][col].median()
print(region, col, 'is ', result/scale)

Europe coef_heat is  8.1737798383928


# Heat Waves

- 9691 Kolkata 1998
- 2046 Paris 2003
- 4417, Aleppo 2010

In [61]:
def make_data(dir_in, geog, location):
    """Function makes data to plot daily city-level HI Max and average
    Args:
        dir_in = directory to get data
        geog = column for geography, city-level = 'ID_HDC_G0'
        location = usually a city id
    """
    
    fn_list = sorted(glob.glob(dir_in+'*.csv')) # get data
    df_out = pd.DataFrame() # to write dataframe
    
     # get leap year cols from 2016
    hi16 = pd.read_csv(fn_list[33]) 
    cols = list(hi16.iloc[:,3:].columns)
    cols = [year[5:] for year in cols] # cols for data frame
    
    temp_list = [] # empty list for temps
    
    # loop through dir and get data
    for i, fn in enumerate(fn_list):
        df = pd.read_csv(fn) # open data frame
        year_label = [(df.columns[3]).split('.')[0]] # get year
        row = df[df[geog] == location]
        temp = row.iloc[:,3:] # get only temp columns
        
        # add in col for leap years
        if temp.shape[1] == 365:
            temp.insert(loc = 59, column = year_label[0]+'.02.29', value = np.nan, allow_duplicates=False)

        # Set Index & Columns
        temp.index = year_label
        temp.columns = cols # revalue to m.d
    
        # add to list
        temp_list.append(temp)
    
    df_out = pd.concat(temp_list) # make one big dataframe
    
    return df_out

In [62]:
def plot_data(df, year, start, end):#, start, end):
    """ Make the data for a plot
    Args: 
        df = df w/ daily HI max for a given city
        year = year you want to plot against average
        start = start of plot in julian days (e.g 1 - 365/366)
        end = end of plot in julian days
    """

    # Deal with leap year
    if year % 4 !=0:
        df.drop(columns ='02.29', inplace = True)
    
    # Subset data
    start = start - 1 # zero indexing 
    subset = df.iloc[:,start:end]
    
    # HI Max for year
    hi_year = subset.loc[str(year)]
    
    # make 34-avg daily hi
    means = subset.mean(axis = 0)
    
    # make colums to date time
    cols = pd.to_datetime([str(year)+'.'+date for date in hi_year.index])
    
    return hi_year, means, cols

In [ ]:
# Find Heat Wave From All DATA
def select_city_year(df, city_id, year):
    "Quick search to find city and years within HI_STATS"
    df_out = df[(df['ID_HDC_G0'] == city_id) & (df['year'] == year)]
    
    return df_out

# open heat events data
events_fn = '/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v2/HI461_1D_STATS.json'
events = pd.read_json(events_fn, orient = 'split')

# [4417, 'Aleppo'] [2046, 'Paris'] [9691, 'Kolkata'] 
city = select_city_year(events, 4417, 2010)
city

In [ ]:
# Select event and compare to long term means
# Kolkata 1998 UID-2920905, Aleppo 2010 
event_id = 'UID-2920905'
event = city[city['UID'] == event_id]
tmax = list(event['tmax'])[0]
dates = list(event['event_dates'])[0]

In [ ]:
#### Heat Index Data
data = 'HI'
DATA_IN = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/'+data+'/' # output from avg temp
t = 46.1

In [ ]:
# Get long term means
# Args
#[4417, 'Aleppo'] 2010 [2046, 'Paris'] 2003 [9691, 'Kolkata'] 1998 ['6955, Dehli']

# Args
city_list = [4417, 'Aleppo'] 
year = 2010

# April 1 to Sep 30 (Use Julian Days), or 1 - 182 lagos
start = 91 
end = 273

# Make Data 
df = make_data(DATA_IN, 'ID_HDC_G0', city_list[0])
years, means, cols = plot_data(df, year, start, end)

In [ ]:
cols

In [ ]:
# make a df
df = pd.DataFrame()
df['dates'] = cols
df['max']  = years.to_list()
df['avg'] = means.to_list()
df['dif'] = df['max'] - df['avg']

In [ ]:
type(df['dates'][0])

In [ ]:
type(dates[0][0])

In [ ]:
# start and end of heat wave
start = dates[0]
end = dates[-1]

In [ ]:
out = df[(df['dates'] >= start) & (df['dates'] <= end)]

In [ ]:
out.max()

In [ ]:
out[out['dif'] == out['dif'].max()]

# Phoenix Caclulations
1990-6-26 Phoenix, AZ Weather History. https://www.wunderground.com/history/daily/KPHX/date/1990-6-26. Weather Underground. (June 10, 2021).<br>A. Minkler, At 118 degrees, Thursday heat in Phoenix breaks daily record set in 1934. The Arizona Republic (2020) (June 18, 2021).


In [63]:
# Functions
def c_to_f(C):
    "Convert HI C to F"
    return 1.8 * C + 32 

def hi_to_wbgt(HI):
    """ Convert HI to WBGT using emprical relationship from Bernard and Iheanacho 2015
    WBGT [◦C] = −0.0034 HI2 + 0.96 HI−34; for HI [◦F]
    """
    
    WBGT = -0.0034*HI**2 + 0.96*HI - 34
    
    return WBGT

In [ ]:
# tmax was 122, rh was 11 using https://www.wpc.ncep.noaa.gov/html/heatindex.shtml
# then HI was 49 

# Check out cities

In [ ]:
def select_city_year(df, city_id, year):
    "Quick search to find city and years within HI_STATS"
    df_out = df[(df['ID_HDC_G0'] == city_id) & (df['year'] == year)]
    
    return df_out


In [ ]:
city = select_city_year(ALL_DATA, 4417, 2010)
city

# LA

In [ ]:
from glob import glob 
import os

In [ ]:
dir_list = '/home/cascade/projects/UrbanHeat/data/interim/ERA5_HI/'


In [ ]:
fn = 'GHS-ERA5-HI_2009.csv'
data = pd.read_csv(dir_list+fn)
city = data[data['ID_HDC_G0'] == 14]

In [ ]:
temps_list = []
dates_list = []
for fn in sorted(os.listdir(dir_list)):
    data = pd.read_csv(dir_list+fn)
    city = data[data['ID_HDC_G0'] == 14]
    dates = list(city.iloc[:,3:])
    temps = list(city.iloc[:,3:].values[0])
    dates_list.extend(dates)
    temps_list.extend(temps)

In [ ]:
len(dates_list)

# Africa

In [ ]:
#### Dependencies
import glob
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import rcParams
import matplotlib.dates as mdates

In [ ]:
def make_data(dir_in, geog, location):
    """Function makes data to plot daily city-level HI Max and average
    Args:
        dir_in = directory to get data
        geog = column for geography, city-level = 'ID_HDC_G0'
        location = usually a city id
    """
    
    fn_list = sorted(glob.glob(dir_in+'*.csv')) # get data
    df_out = pd.DataFrame() # to write dataframe
    
     # get leap year cols from 2016
    hi16 = pd.read_csv(fn_list[33]) 
    cols = list(hi16.iloc[:,3:].columns)
    cols = [year[5:] for year in cols] # cols for data frame
    
    temp_list = [] # empty list for temps
    
    # loop through dir and get data
    for i, fn in enumerate(fn_list):
        df = pd.read_csv(fn) # open data frame
        year_label = [(df.columns[3]).split('.')[0]] # get year
        row = df[df[geog] == location]
        temp = row.iloc[:,3:] # get only temp columns
        
        # add in col for leap years
        if temp.shape[1] == 365:
            temp.insert(loc = 59, column = year_label[0]+'.02.29', value = np.nan, allow_duplicates=False)

        # Set Index & Columns
        temp.index = year_label
        temp.columns = cols # revalue to m.d
    
        # add to list
        temp_list.append(temp)
    
    df_out = pd.concat(temp_list) # make one big dataframe
    
    return df_out

In [ ]:
def plot_data(df, year, start, end):#, start, end):
    """ Make the data for a plot
    Args: 
        df = df w/ daily HI max for a given city
        year = year you want to plot against average
        start = start of plot in julian days (e.g 1 - 365/366)
        end = end of plot in julian days
    """

    # Deal with leap year
    if year % 4 !=0:
        df.drop(columns ='02.29', inplace = True)
    
    # Subset data
    start = start - 1 # zero indexing 
    subset = df.iloc[:,start:end]
    
    # HI Max for year
    hi_year = subset.loc[str(year)]
    
    # make 34-avg daily hi
    means = subset.mean(axis = 0)
    
    # make colums to date time
    cols = pd.to_datetime([str(year)+'.'+date for date in hi_year.index])
    
    return hi_year, means, cols

In [ ]:
# Args
#[3342, 'Cairo'] #[1910, 'Accra'] 3268, Cape Town
#[4417, 'Aleppo'] 2010 [2046, 'Paris'] 2003 [9691, 'Kolkata'] 1998 ['6955, Dehli']

# Args
city_list = [9691, 'Kolkata'] 
year = 1998
font_size = 10


# April 1 to Sep 30 (Use Julian Days), or 1 - 182 lagos
start = 91 
end = 273

# Labels
hi_label = str(year)+' '+data
labels =  ['avg. '+data, hi_label, str(t)+''+DS+'C']  # <<<<<------------ Be sure to update! 

# Make Data 
df = make_data(DATA_IN, 'ID_HDC_G0', city_list[0])
years, means, cols = plot_data(df, year, start, end)

# Trend First vs. Second Half of Data

In [ ]:
data.head()

In [ ]:
#### Share of exposure due to heat by 17 year split

## 1983 - 1999
data1 = HI_STATS[(HI_STATS['year'] >= 1983) & (HI_STATS['year'] < 2000)]
coef1pop , r21pop, p1pop  = lm_func(data1 , 'people_days_pop')
coef1heat , r21heat, p1heat = lm_func(data1 , 'people_days_heat')

years = list(np.unique(data1['year']))
plt.plot(years, data1.groupby('year')['people_days_heat'].sum())
sns.regplot(years, data1.groupby('year')['people_days_heat'].sum(), 
            color = 'blue', scatter = False, truncate = True)

## 2000 - 2016
data2 = HI_STATS[(HI_STATS['year'] >= 2000) & (HI_STATS['year'] <= 2016)]
coef2heat , r22heat, p2heat = lm_func(data2 , 'people_days_heat')
coef2pop , r22pop, p1pop  = lm_func(data2 , 'people_days_pop')

years = list(np.unique(data1['year']))
plt.plot(years, data2.groupby('year')['people_days_heat'].sum())
sns.regplot(years, data2.groupby('year')['people_days_heat'].sum(), 
            color = 'orange', scatter = False, truncate = True)

In [ ]:
## 2000 - 2016
data2pop = HI_STATS[(HI_STATS['year'] >= 1983) & (HI_STATS['year'] < 2000)]
coef2pop , r22pop, p1pop  = lm_func(data2pop , 'people_days_pop')

data2heat = HI_STATS[(HI_STATS['year'] >= 2000) & (HI_STATS['year'] <= 2016)]
coef2heat , r22heat, p2heat = lm_func(data2heat , 'people_days_heat')


In [ ]:
## Estimates
print('From 83 - 99, contribution from heat was', coef1heat/(coef1pop+coef1heat))
print('From 00 - 16, contribution from heat was', coef2heat/(coef2pop+coef2heat))
print('From 83 - 00, heat was', coef1heat/10**9, round(p1heat, 3))
print('From 00 - 16, heat was', coef2heat/10**9)
print('From 83 - 00, pop was', coef1pop/10**9)
print('From 00 - 16, pop was', coef2pop/10**9)